In [108]:
import os
import shutil

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.features import geometry_mask
import rioxarray
import scipy
from shapely.geometry import shape, Polygon
import xrspatial

from valleys.utils import setup_wbt
from valleys.utils import get_stream_raster_from_hillslope
from valleys.utils import vectorize_stream
from valleys.cross_section import preprocess_channel
from valleys.cross_section import get_cross_section_points
from valleys.terrain import get_terrain_features
from valleys.breakpoints import get_break_points
from valleys.breakpoints import map_valley_floor

In [115]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
wbt = setup_wbt(os.path.expanduser("~/opt/WBT/"), os.path.abspath("../data/wb_outputs/"))

In [103]:
def map_floor(wbt, dem, hs, simple_threshold=15, 
              xs_spacing=20, xs_width=400, xs_point_spacing=10, 
              peak_threshold=0.003, slope_threshold_bp =0.25, max_height=100,
              quantile = 0.6, buffer=0, slope_threshold=20):

    stream_binary = get_stream_raster_from_hillslope(wbt, hs)
    stream = vectorize_stream(wbt, stream_binary)
    centerline = preprocess_channel(stream, method='simplify', threshold=simple_threshold).iloc[0]
    points = get_cross_section_points(centerline, xs_spacing=xs_spacing, xs_width=xs_width, xs_point_spacing=xs_point_spacing)
    points, slope, curvature, hand = get_terrain_features(wbt, points, dem, hs)
    break_points = get_break_points(points, peak_threshold=peak_threshold, slope_threshold=slope_threshold_bp)
    break_points = break_points.loc[break_points['hand'] < max_height]

    floor = map_valley_floor(break_points, hand, slope, quantile=0.6, buffer=0, slope_threshold=20)

    return points, break_points, slope, curvature, hand, floor

## Load in test valley

In [107]:
# 1801, 12
# 1807, 4
# 1804, 22

dem1 = rioxarray.open_rasterio(os.path.abspath("../data/huc12s/180101100904/subbasins/12.0_dem.tif")).squeeze()
dem2 = rioxarray.open_rasterio(os.path.abspath("../data/huc12s/180701020604/subbasins/4.0_dem.tif")).squeeze()
dem3 = rioxarray.open_rasterio(os.path.abspath("../data/huc12s/180400060403/subbasins/22.0_dem.tif")).squeeze()

hs1 = rioxarray.open_rasterio(os.path.abspath("../data/huc12s/180101100904/subbasins/12.0_hillslopes.tif")).squeeze()
hs2 = rioxarray.open_rasterio(os.path.abspath("../data/huc12s/180701020604/subbasins/4.0_hillslopes.tif")).squeeze()
hs3 = rioxarray.open_rasterio(os.path.abspath("../data/huc12s/180400060403/subbasins/22.0_hillslopes.tif")).squeeze() 

## Make Figures

In [121]:
#if os.path.isdir("../data/figures/"):
#    shutil.rmtree("../data/figures/")
#os.mkdir("../data/figures/")

for i, (dem, hs) in enumerate(zip([dem1, dem2, dem3], [hs1, hs2, hs3])):
    if i != 1:
        continue
    points, break_points, slope, curvature, hand, floor = map_floor(wbt, dem, hs)

    hillshade = xrspatial.hillshade(dem)

    fig, ax = plt.subplots(figsize=(12,12))
    slope.plot(ax=ax)
    break_points.plot(ax=ax, color='pink', markersize=3)
    floor.dissolve().plot(ax=ax, facecolor='none', edgecolor='blue')
    plt.savefig(f"../data/figures/{i}_slope.png")
    plt.ioff()

    fig, ax = plt.subplots(figsize=(12,12))
    hillshade.plot(ax=ax, cmap='gray')
    break_points.plot(ax=ax, color='pink', markersize=3)
    floor.dissolve().plot(ax=ax, facecolor='none', edgecolor='blue')
    plt.savefig(f"../data/figures/{i}_hillshade.png")
    plt.ioff()

/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/valleys-apqonAG6-py3.10/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/valleys-apqonAG6-py3.10/lib/python3.10/site-packages/xrspatial/hillshade.py:23: RuntimeWarning: invalid value encountered in sin
  shaded = np.sin(altituderad) * np.sin(slope) + \
/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/valleys-apqonAG6-py3.10/lib/python3.10/site-packages/xrspatial/hillshade.py:24: RuntimeWarning: invalid value encountered in cos
  np.cos(altituderad) * np.cos(slope) * \
/Users/arthurkoehl/Library/Caches/pypoetry/virtualenvs/valleys-apqonAG6-py3.10/lib/python3.10/site-packages/xrspatial/hillshade.py:25: RuntimeWa